In [1]:
import cv2,os
data_path='dataset/dataset'
data_categories=os.listdir(data_path)
labels=[i for i in range(len(data_categories))]
label_dict=dict(zip(data_categories,labels))
print(label_dict)
print(data_categories)
print(labels)

{'with_mask': 0, 'without_mask': 1}
['with_mask', 'without_mask']
[0, 1]


In [2]:
image_size = 120
image_data=[]
target=[]

for data_category in data_categories:
  folder_path=os.path.join(data_path,data_category)
  image_names=os.listdir(folder_path)

  for image_name in image_names:
    image_path=os.path.join(folder_path,image_name)
    image=cv2.imread(image_path)

    try:
      #Converting image to a single color i.e gray scale
      gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
      #resizing a image to size 120x120 since we need a fixed size for all image to work on
      resized=cv2.resize(gray,(image_size,image_size))
      image_data.append(resized)
      #appending the image and the label(categorized) into the list
      target.append(label_dict[data_category])

      #While parsing through images if any image is not found or anything else exception is generated
      #It gets caught and without stopping the loop it continnues to the next image
    except Exception as k:
      print('Exception:',k)

In [3]:
import numpy as np

image_data=np.array(image_data)/255.0
image_data=np.reshape(image_data,(image_data.shape[0],image_size,image_size,1))
target=np.array(target)


from keras.utils import np_utils


target= np_utils.to_categorical(target)

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.callbacks import ModelCheckpoint

model=Sequential()

#1st Convolutional layer
model.add(Conv2D(200,(3,3),input_shape=image_data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#2nd Convolutional layer
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flattening Layer to stack the output from the 2nd Convolutional layer
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
#Final layer with two output for 2 categories
model.add(Dense(2,activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])



In [5]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(image_data,target,test_size=0.1)

In [6]:
history=model.fit(train_data,train_target,epochs=20,validation_split=0.2)

Epoch 1/20
87/87 [==============================] - 316s 4s/step - loss: 0.6895 - accuracy: 0.5970 - val_loss: 0.6025 - val_accuracy: 0.6565
Epoch 2/20
87/87 [==============================] - 320s 4s/step - loss: 0.5747 - accuracy: 0.7024 - val_loss: 0.5608 - val_accuracy: 0.7420
Epoch 3/20
87/87 [==============================] - 315s 4s/step - loss: 0.5430 - accuracy: 0.7191 - val_loss: 0.4985 - val_accuracy: 0.7580
Epoch 4/20
87/87 [==============================] - 315s 4s/step - loss: 0.3971 - accuracy: 0.8246 - val_loss: 0.3264 - val_accuracy: 0.8623
Epoch 5/20
87/87 [==============================] - 316s 4s/step - loss: 0.2953 - accuracy: 0.8782 - val_loss: 0.2933 - val_accuracy: 0.8797
Epoch 6/20
87/87 [==============================] - 314s 4s/step - loss: 0.2486 - accuracy: 0.8949 - val_loss: 0.3207 - val_accuracy: 0.8638
Epoch 7/20
87/87 [==============================] - 320s 4s/step - loss: 0.2083 - accuracy: 0.9177 - val_loss: 0.2822 - val_accuracy: 0.8812
Epoch 8/20
87